In [1]:
import pandas as pd

In [2]:
#загрузим данные
taxi = pd.read_csv('2_taxi_nyc.csv')

In [5]:
taxi.head()

,pickup_dt,pickup_month,borough,pickups,hday,spd,vsb,temp,dewp,slp,pcp 01,pcp 06,pcp 24,sd
0,2015-01-01 01:00:00,Jan,Bronx,152,Y,5.0,10.0,30.0,7.0,1023.5,0.0,0.0,0.0,0.0
1,2015-01-01 01:00:00,Jan,Brooklyn,1519,Y,5.0,10.0,30.0,7.0,1023.5,0.0,0.0,0.0,0.0
2,2015-01-01 01:00:00,Jan,EWR,0,Y,5.0,10.0,30.0,7.0,1023.5,0.0,0.0,0.0,0.0
3,2015-01-01 01:00:00,Jan,Manhattan,5258,Y,5.0,10.0,30.0,7.0,1023.5,0.0,0.0,0.0,0.0
4,2015-01-01 01:00:00,Jan,Queens,405,Y,5.0,10.0,30.0,7.0,1023.5,0.0,0.0,0.0,0.0


In [3]:
#узнаем размер датасета
taxi.shape

(29101, 14)

In [4]:
#посмотрим на типы колонок
taxi.dtypes

pickup_dt        object
pickup_month     object
borough          object
pickups           int64
hday             object
spd             float64
vsb             float64
temp            float64
dewp            float64
slp             float64
pcp 01          float64
pcp 06          float64
pcp 24          float64
sd              float64
dtype: object

In [6]:
#заменим пробел в названиях колонок на нижнее подчеркивание
taxi.rename(columns={'pcp 01': 'pcp_01', 'pcp 06': 'pcp_06', 'pcp 24': 'pcp_24'}, inplace=True)

In [8]:
#cколько раз в данных встречается район Бруклин?
taxi.borough.value_counts()

Bronx            4343
Brooklyn         4343
EWR              4343
Manhattan        4343
Queens           4343
Staten Island    4343
Name: borough, dtype: int64

In [9]:
#общее количество поездок
taxi.pickups.sum()

14265773

In [12]:
#район с наибольшим количеством поездок за весь период
taxi.groupby('borough', as_index=False) \
    .agg({'pickups': 'sum'}) \
    .sort_values('pickups', ascending=False)

,borough,pickups
3,Manhattan,10367841
1,Brooklyn,2321035
4,Queens,1343528
0,Bronx,220047
5,Staten Island,6957
2,EWR,105


In [18]:
#район с наименьшим числом поездок
min_pickups = taxi.groupby('borough') \
                  .agg({'pickups': 'sum'}) \
                  .idxmin()

min_pickups

pickups    EWR
dtype: object

In [20]:
#районы, из которых по праздникам в среднем поступает больше заказов, чем в обычные дни
days = taxi.groupby(['borough', 'hday'], as_index=False) \
           .agg({'pickups': 'mean'}) \
           .pivot(index='borough', columns='hday', values='pickups')

days.Y > days.N

borough
Bronx            False
Brooklyn         False
EWR               True
Manhattan        False
Queens            True
Staten Island    False
dtype: bool

In [22]:
#число поездок по месяцам для каждого района по убыванию
pickups_by_mon_bor = taxi.groupby(['borough', 'pickup_month'], as_index=False) \
                         .agg({'pickups': 'sum'}) \
                         .sort_values('pickups', ascending=False)

In [24]:
#функция, которая получает на вход колонку с температурой в °F, и возвращает значения, переведенные в градусы Цельсия
def temp_to_celcius(temp):
    return (temp - 32) * 5.0 / 9.0

temp_to_celcius(taxi.temp)

0        -1.111111
1        -1.111111
2        -1.111111
3        -1.111111
4        -1.111111
           ...    
29096    23.888889
29097    23.888889
29098    23.888889
29099    23.888889
29100    23.888889
Name: temp, Length: 29101, dtype: float64